In [ ]:
from pathlib import Path

from gretel_trainer.relational.connectors import Connector
from gretel_trainer.relational.extractor import ExtractorConfig, TableExtractor
from sqlalchemy import create_engine

# Where we are gonna drop these tables!
storage_dir = Path("table-data")
storage_dir.mkdir(exist_ok=True)

!curl -o "ecom.db" "https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/rdb/ecom_xf.db"

In [ ]:
connector = Connector.from_conn_str("sqlite:///ecom.db")

# Change this if you want more/less subsetting. You can set it to 0 and only the headers
# of the tables will be extracted. The default mode is -1 (get all the tables)
#
# If you set to a value between 0..1 (exclusive), then that will be the rough
# percentage of rows that are sampled. So a value of .5 will sample roughly half.

config = ExtractorConfig(
    target_row_count=100,
)

extractor = TableExtractor(connector=connector, config=config, storage_dir=storage_dir)
extractor.sample_tables()

In [ ]:
# All the tables are on disk in the `storage_dir`, you can load one
# back in as a DF just based on the table name.

import random
import pandas as pd

df = extractor.get_table_df(random.choice(extractor.table_order))
print(df.head())